In [4]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("20240627").getOrCreate()
# winter_df = spark.read.csv("hdfs:////encore/tpss_bcycl_od_statnhm_202001.csv", encoding='cp949', header=True, schema=schema)

24/06/27 06:20:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
spark

In [7]:
pwd

'/home/hadoop/workspace'

In [8]:
winter_df = spark.read.csv("file:///home/hadoop/workspace/tpss_bcycl_od_statnhm_202001.csv", 
               encoding='cp949', header=True)


In [9]:
from pyspark.sql.types import StructType, IntegerType, StringType, FloatType
schema = StructType().\
      add('기준_날짜',  StringType(), True ).\
        add('구분코드',  StringType(), True ).\
        add('기준_시간',  StringType(), True ).\
        add('시작_대여소ID',  StringType(), True ).\
        add('시작_대여소명',  StringType(), True ).\
        add('종료_대여소ID',  StringType(), True ).\
        add('종료_대여소명',  StringType(), True ).\
        add('전체건수',  IntegerType(), True ).\
        add('전체이용시간(분)',  IntegerType(), True ).\
        add('전체이용거리(m)' , FloatType(), True )


In [10]:
winter_df.printSchema()

root
 |-- 기준_날짜: string (nullable = true)
 |-- 구분코드: string (nullable = true)
 |-- 기준_시간: string (nullable = true)
 |-- 시작_대여소ID: string (nullable = true)
 |-- 시작_대여소명: string (nullable = true)
 |-- 종료_대여소ID: string (nullable = true)
 |-- 종료_대여소명: string (nullable = true)
 |-- 전체건수: string (nullable = true)
 |-- 전체이용시간(분): string (nullable = true)
 |-- 전체이용거리(m): string (nullable = true)



In [11]:
winter_df = spark.read.format("csv").\
            option("header", True).\
            option("encoding", "cp949").\
            schema(schema).csv("file:///home/hadoop/workspace/tpss_bcycl_od_statnhm_202001.csv")


In [12]:
summer = spark.read.format("csv").\
            option("header", True).\
            option("encoding", "cp949").\
            schema(schema).csv("file:///home/hadoop/workspace/tpss_bcycl_od_statnhm_202007.csv")


In [13]:
summer.printSchema()

root
 |-- 기준_날짜: string (nullable = true)
 |-- 구분코드: string (nullable = true)
 |-- 기준_시간: string (nullable = true)
 |-- 시작_대여소ID: string (nullable = true)
 |-- 시작_대여소명: string (nullable = true)
 |-- 종료_대여소ID: string (nullable = true)
 |-- 종료_대여소명: string (nullable = true)
 |-- 전체건수: integer (nullable = true)
 |-- 전체이용시간(분): integer (nullable = true)
 |-- 전체이용거리(m): float (nullable = true)



In [14]:
import pyspark.sql.functions as fn 
summer.agg(*[
    fn.count(fn.when(fn.isnull(x), x)).\
             alias(x) 
             for x in summer.columns 
]).show()



winter_df= winter_df.withColumn("기준_날짜", fn.to_date(fn.col("기준_날짜"), "yyyyMMdd"))


+---------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+
|기준_날짜|구분코드|기준_시간|시작_대여소ID|시작_대여소명|종료_대여소ID|종료_대여소명|전체건수|전체이용시간(분)|전체이용거리(m)|
+---------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+
|        0|       0|        0|            0|            0|            0|            0|       0|               0|              0|
+---------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+



In [15]:
winter_df= winter_df.withColumn("기준_날짜", fn.to_date(fn.col("기준_날짜"), "yyyyMMdd"))

In [16]:
winter_df.first()

Row(기준_날짜=datetime.date(2020, 1, 1), 구분코드='0', 기준_시간='0000', 시작_대여소ID='ST-443', 시작_대여소명='성산2동_041_2', 종료_대여소ID='ST-82', 종료_대여소명='성산2동_041_3', 전체건수=1, 전체이용시간(분)=2, 전체이용거리(m)=450.0)

In [17]:
winter_df = winter_df.withColumn("요일", fn.date_format(fn.col("기준_날짜"), "E"))

In [18]:
winter_df.show()

+----------+--------+---------+-------------+----------------+-------------+----------------+--------+----------------+---------------+----+
| 기준_날짜|구분코드|기준_시간|시작_대여소ID|   시작_대여소명|종료_대여소ID|   종료_대여소명|전체건수|전체이용시간(분)|전체이용거리(m)|요일|
+----------+--------+---------+-------------+----------------+-------------+----------------+--------+----------------+---------------+----+
|2020-01-01|       0|     0000|       ST-443|   성산2동_041_2|        ST-82|   성산2동_041_3|       1|               2|          450.0| Wed|
|2020-01-01|       0|     0000|        ST-56|   여의동_005_10|      ST-1321|  영등포동_036_1|       1|              10|         1460.0| Wed|
|2020-01-01|       0|     0000|      ST-1546|    조원동_004_1|       ST-703|    신사동_023_1|       1|               5|          700.0| Wed|
|2020-01-01|       1|     0000|      ST-1701|   당산2동_065_1|      ST-1701|   당산2동_065_1|       1|              28|        70670.0| Wed|
|2020-01-01|       1|     0000|      ST-1390|   수유2동_030_1|      ST-1101|    미아동_010_1|    

In [19]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

# Create DataFrame
schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)
df.groupBy("department").count().show(truncate=False)


root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)



+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+

+----------+-----+
|department|count|
+----------+-----+
|Sales     |3    |
|Finance   |4    |
|Marketing |2    |
+----------+-----+



In [20]:
winter_df.printSchema()

root
 |-- 기준_날짜: date (nullable = true)
 |-- 구분코드: string (nullable = true)
 |-- 기준_시간: string (nullable = true)
 |-- 시작_대여소ID: string (nullable = true)
 |-- 시작_대여소명: string (nullable = true)
 |-- 종료_대여소ID: string (nullable = true)
 |-- 종료_대여소명: string (nullable = true)
 |-- 전체건수: integer (nullable = true)
 |-- 전체이용시간(분): integer (nullable = true)
 |-- 전체이용거리(m): float (nullable = true)
 |-- 요일: string (nullable = true)



In [21]:
winter_df.groupby("요일").count().orderBy("count",ascending=True).show(truncate=False)

+----+------+
|요일|count |
+----+------+
|Sun |170925|
|Sat |184020|
|Mon |185929|
|Tue |197435|
|Wed |289054|
|Fri |305148|
|Thu |336492|
+----+------+



In [22]:
summer.printSchema()

root
 |-- 기준_날짜: string (nullable = true)
 |-- 구분코드: string (nullable = true)
 |-- 기준_시간: string (nullable = true)
 |-- 시작_대여소ID: string (nullable = true)
 |-- 시작_대여소명: string (nullable = true)
 |-- 종료_대여소ID: string (nullable = true)
 |-- 종료_대여소명: string (nullable = true)
 |-- 전체건수: integer (nullable = true)
 |-- 전체이용시간(분): integer (nullable = true)
 |-- 전체이용거리(m): float (nullable = true)



In [23]:
summer= summer.withColumn("기준_날짜", fn.to_date(fn.col("기준_날짜"), "yyyyMMdd"))
summer = summer.withColumn("요일", fn.date_format(fn.col("기준_날짜"), "E"))

In [24]:
summer.limit(1).show()

+----------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+----+
| 기준_날짜|구분코드|기준_시간|시작_대여소ID|시작_대여소명|종료_대여소ID|종료_대여소명|전체건수|전체이용시간(분)|전체이용거리(m)|요일|
+----------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+----+
|2020-07-01|       1|     0000|       ST-883| 조원동_034_1|      ST-1265| 조원동_014_1|       1|               7|            0.0| Wed|
+----------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+----+



In [25]:
summer.groupby("요일").count().orderBy("count",ascending=True).show(truncate=False)
# winter_df.groupby("요일").count().sort('count').show()

+----+------+
|요일|count |
+----+------+
|Mon |466809|
|Sun |597415|
|Tue |682796|
|Sat |747857|
|Wed |781305|
|Thu |820097|
|Fri |914310|
+----+------+



In [26]:
summer_fri = summer.filter(fn.col("요일")=="Fri")

In [27]:
summer_fri.first()

Row(기준_날짜=datetime.date(2020, 7, 3), 구분코드='0', 기준_시간='0000', 시작_대여소ID='ST-1037', 시작_대여소명='갈현1동_044_1', 종료_대여소ID='ST-460', 종료_대여소명='응암3동_017_1', 전체건수=1, 전체이용시간(분)=24, 전체이용거리(m)=0.0, 요일='Fri')

In [28]:
summer_fri.select(fn.col("기준_시간")).distinct().sort("기준_시간").show()

+---------+
|기준_시간|
+---------+
|     0000|
|     0005|
|     0010|
|     0015|
|     0020|
|     0025|
|     0030|
|     0035|
|     0040|
|     0045|
|     0050|
|     0055|
|     0100|
|     0105|
|     0110|
|     0115|
|     0120|
|     0125|
|     0130|
|     0135|
+---------+
only showing top 20 rows



In [29]:
summer_fri.groupby("기준_시간").count().orderBy("count",ascending=False).show()

+---------+-----+
|기준_시간|count|
+---------+-----+
|     1805| 8694|
|     1810| 8636|
|     1815| 8265|
|     1820| 7980|
|     1825| 7753|
|     1835| 7568|
|     1830| 7556|
|     1840| 7508|
|     1845| 7454|
|     1800| 7451|
|     1850| 7401|
|     1855| 7297|
|     1905| 7036|
|     1900| 7027|
|     1910| 6727|
|     1915| 6664|
|     1755| 6629|
|     1920| 6486|
|     1750| 6427|
|     1745| 6399|
+---------+-----+
only showing top 20 rows



In [30]:
summer_fri_peek = summer_fri.filter((fn.col("기준_시간")>="1800") & (fn.col("기준_시간")<="1900") )

In [31]:
summer_fri_peek.show()

+----------+--------+---------+-------------+----------------+-------------+----------------+--------+----------------+---------------+----+
| 기준_날짜|구분코드|기준_시간|시작_대여소ID|   시작_대여소명|종료_대여소ID|   종료_대여소명|전체건수|전체이용시간(분)|전체이용거리(m)|요일|
+----------+--------+---------+-------------+----------------+-------------+----------------+--------+----------------+---------------+----+
|2020-07-03|       0|     1800|       ST-506|성수2가3동_021_2|      ST-1660|   자양3동_025_1|       1|              12|        1490.58| Fri|
|2020-07-03|       0|     1800|      ST-2206| 북가좌1동_019_1|       ST-210|   성산1동_009_1|       1|               9|            0.0| Fri|
|2020-07-03|       0|     1800|       ST-231|   홍은2동_036_1|       ST-231|   홍은2동_036_1|       1|              64|         8920.0| Fri|
|2020-07-03|       0|     1800|        ST-57|   여의동_005_11|      ST-1751|    수색동_031_1|       1|              46|            0.0| Fri|
|2020-07-03|       0|     1800|      ST-1694|   화곡3동_003_1|      ST-1064|   발산1동_060_1|     

In [32]:
temp = summer_fri.filter((fn.col("기준_시간")>="1200") & (fn.col("기준_시간")<="1300") )

In [33]:
temp.select(fn.mean(fn.col('전체이용거리(m)')).alias("평균거리"),
                      fn.median(fn.col('전체이용거리(m)')).alias("중앙값"),
                       fn.max(fn.col('전체이용거리(m)')).alias("최댓값"),
                      ).show()


+------------------+------+--------+
|          평균거리|중앙값|  최댓값|
+------------------+------+--------+
|1601.6666854104033|   0.0|226480.0|
+------------------+------+--------+



In [34]:
summer.groupby("시작_대여소명","종료_대여소명").count().sort("count",ascending=False).show(truncate=False)

+----------------+----------------+-----+
|시작_대여소명   |종료_대여소명   |count|
+----------------+----------------+-----+
|자양3동_050_1   |자양3동_050_1   |4637 |
|여의동_009_1    |여의동_009_1    |4400 |
|망원2동_031_1   |망원2동_031_1   |3816 |
|신림동_029_1    |신림동_029_1    |3555 |
|옥수동_036_1    |옥수동_036_1    |3301 |
|마장동_032_1    |마장동_032_1    |2932 |
|독산1동_031_2   |독산1동_031_2   |2819 |
|당산2동_017_1   |당산2동_017_1   |2759 |
|흑석동_028_1    |흑석동_028_1    |2537 |
|반포3동_001_2   |반포3동_001_2   |2483 |
|응봉동_026_1    |응봉동_026_1    |2467 |
|성산2동_078_2   |성산2동_078_2   |2441 |
|삼성동_003_1    |삼성동_003_1    |2354 |
|조원동_004_1    |조원동_004_1    |2285 |
|잠원동_005_1    |잠원동_005_1    |2255 |
|응암1동_032_1   |응암1동_032_1   |2245 |
|노량진1동_001_3 |노량진1동_001_3 |2239 |
|성수1가1동_014_3|성수1가1동_014_3|2122 |
|오륜동_001_4    |오륜동_001_4    |2037 |
|홍은2동_001_1   |홍은2동_001_1   |1905 |
+----------------+----------------+-----+
only showing top 20 rows



In [35]:
summer_fri_peek.select(fn.mean(fn.col('전체이용시간(분)')).alias("평균"),
                      fn.median(fn.col('전체이용시간(분)')).alias("중앙값"),
                       fn.max(fn.col('전체이용시간(분)')).alias("최댓값"),
                       fn.count(fn.col('전체이용시간(분)')).alias("count")
                      ).show()


# summer_fri_0000.select(fn.mean(fn.col('전체이용시간(분)')).alias("평균"),
#                       fn.median(fn.col('전체이용시간(분)')).alias("중앙값"),
#                        fn.max(fn.col('전체이용시간(분)')).alias("최댓값"),
#                        fn.count(fn.col('전체이용시간(분)')).alias("count")
#                       ).show()





summer.groupby("시작_대여소명", "종료_대여소명").count().sort('count', ascending=True).count()


+------------------+------+------+------+
|              평균|중앙값|최댓값| count|
+------------------+------+------+------+
|29.404135599960235|  18.0|  1580|100590|
+------------------+------+------+------+



334460

In [36]:
summer.withColumn("동", fn.split("시작_대여소명","_")[0]).first()

Row(기준_날짜=datetime.date(2020, 7, 1), 구분코드='1', 기준_시간='0000', 시작_대여소ID='ST-883', 시작_대여소명='조원동_034_1', 종료_대여소ID='ST-1265', 종료_대여소명='조원동_014_1', 전체건수=1, 전체이용시간(분)=7, 전체이용거리(m)=0.0, 요일='Wed', 동='조원동')

In [37]:
summer = summer.withColumn("동", fn.split("시작_대여소명","_")[0])

In [38]:
summer.printSchema()

root
 |-- 기준_날짜: date (nullable = true)
 |-- 구분코드: string (nullable = true)
 |-- 기준_시간: string (nullable = true)
 |-- 시작_대여소ID: string (nullable = true)
 |-- 시작_대여소명: string (nullable = true)
 |-- 종료_대여소ID: string (nullable = true)
 |-- 종료_대여소명: string (nullable = true)
 |-- 전체건수: integer (nullable = true)
 |-- 전체이용시간(분): integer (nullable = true)
 |-- 전체이용거리(m): float (nullable = true)
 |-- 요일: string (nullable = true)
 |-- 동: string (nullable = true)



In [39]:
summer.select("시작_대여소ID","동").distinct().groupby("동").count().sort("count",ascending=False).show()

+------------------+-----+
|                동|count|
+------------------+-----+
|            여의동|   34|
|           가양1동|   33|
|            상암동|   27|
|            진관동|   25|
|           오류2동|   25|
|종로1・2・3・4가동|   24|
|            가산동|   18|
|           발산1동|   17|
|           독산1동|   17|
|           양재1동|   15|
|            내곡동|   15|
|          한강로동|   14|
|            신촌동|   14|
|           문정2동|   14|
|           성산2동|   13|
|             목1동|   13|
|            사직동|   13|
|           삼성1동|   12|
|           양재2동|   12|
|           역삼1동|   12|
+------------------+-----+
only showing top 20 rows



In [40]:
master = spark.read.csv("file:///home/hadoop/workspace/서울시 따릉이대여소 마스터 정보.csv",
                        encoding='cp949', header=True)

In [41]:
master.first()

Row(대여소_ID='ST-999', 주소1='서울특별시 양천구 목동서로 280', 주소2='목동아파트 8단지 상가동', 위도='0.000000', 경도='0.000000')

In [42]:
new_summer = summer.join(master,summer["시작_대여소ID"]==master["대여소_ID"],"left")

In [43]:
# summer.filter(fn.col("시작_대여소ID")=="ST-1262").show()

In [44]:
master.limit(1).show()

+---------+------------------------------+-----------------------+--------+--------+
|대여소_ID|                         주소1|                  주소2|    위도|    경도|
+---------+------------------------------+-----------------------+--------+--------+
|   ST-999|서울특별시 양천구 목동서로 280|목동아파트 8단지 상가동|0.000000|0.000000|
+---------+------------------------------+-----------------------+--------+--------+



In [45]:
# 구 컬럼 생성
# 구별 따릉이 대여소 개수

In [46]:
new_summer.printSchema()

root
 |-- 기준_날짜: date (nullable = true)
 |-- 구분코드: string (nullable = true)
 |-- 기준_시간: string (nullable = true)
 |-- 시작_대여소ID: string (nullable = true)
 |-- 시작_대여소명: string (nullable = true)
 |-- 종료_대여소ID: string (nullable = true)
 |-- 종료_대여소명: string (nullable = true)
 |-- 전체건수: integer (nullable = true)
 |-- 전체이용시간(분): integer (nullable = true)
 |-- 전체이용거리(m): float (nullable = true)
 |-- 요일: string (nullable = true)
 |-- 동: string (nullable = true)
 |-- 대여소_ID: string (nullable = true)
 |-- 주소1: string (nullable = true)
 |-- 주소2: string (nullable = true)
 |-- 위도: string (nullable = true)
 |-- 경도: string (nullable = true)



In [47]:
new_summer.select("주소1").limit(1).show()

+---------------------------------+
|                            주소1|
+---------------------------------+
|서울특별시 관악구 남부순환로 1369|
+---------------------------------+



In [48]:
new_summer = new_summer.withColumn("구", fn.split("주소1"," ")[1])

In [49]:
new_summer.select("구").limit(10).show()

+------+
|    구|
+------+
|관악구|
|광진구|
|송파구|
|강서구|
|강북구|
|관악구|
|노원구|
|도봉구|
|노원구|
|양천구|
+------+



In [50]:
new_summer.select("시작_대여소ID","구").distinct().groupby("구").count().sort("count",ascending=False).show()

+--------+-----+
|      구|count|
+--------+-----+
|  송파구|  131|
|  강서구|  125|
|  강남구|  118|
|  서초구|  113|
|영등포구|  111|
|  마포구|   97|
|  종로구|   94|
|  노원구|   89|
|  구로구|   87|
|  강동구|   83|
|  양천구|   80|
|  성동구|   77|
|  광진구|   76|
|서대문구|   74|
|  성북구|   74|
|  은평구|   68|
|동대문구|   68|
|    중구|   66|
|  관악구|   65|
|  중랑구|   64|
+--------+-----+
only showing top 20 rows



In [51]:
k = new_summer.filter(fn.col("구") == "강남구")
d = new_summer.filter(fn.col("구") == "도봉구")

In [52]:
k.limit(1).show()

+----------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+----+-------+---------+--------------------------------+-----+---------+----------+------+
| 기준_날짜|구분코드|기준_시간|시작_대여소ID|시작_대여소명|종료_대여소ID|종료_대여소명|전체건수|전체이용시간(분)|전체이용거리(m)|요일|     동|대여소_ID|                           주소1|주소2|     위도|      경도|    구|
+----------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+----+-------+---------+--------------------------------+-----+---------+----------+------+
|2020-07-01|       1|     0000|      ST-1880|도곡2동_042_1|       ST-548|잠실2동_035_1|       1|              79|       10263.59| Wed|도곡2동|  ST-1880|서울특별시 강남구 언주로30길 ...| null|37.488972|127.051826|강남구|
+----------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+----+-------+---------+--------------------------------+-----+---------+---------

In [53]:
pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org numpy

Note: you may need to restart the kernel to use updated packages.


In [54]:
pip install "numpy"

Note: you may need to restart the kernel to use updated packages.


In [55]:
import pyspark.ml.stat as st
from pyspark.ml.feature import VectorAssembler
summary = st.Summarizer.metrics("mean", "std", "count")
assembler = VectorAssembler(inputCols=['전체이용시간(분)'], outputCol="features")


d_vec = assembler.transform(d)
k_vec = assembler.transform(k)


summary1 = d_vec.select(summary.summary(d_vec['features']))
summary2 = k_vec.select(summary.summary(k_vec['features']))


In [57]:
pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 14.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [58]:
new_summer.approxQuantile("전체이용시간(분)", [0.0, 0.25, 0.5, 0.75, 1.0], 0.5)


rdd1 = k_vec.select("전체이용시간(분)").rdd.map(lambda x : x[0])
rdd2 = d_vec.select("전체이용시간(분)").rdd.map(lambda x : x[0])


강남구 = rdd1.collect()
도봉구 = rdd2.collect()


from scipy import stats


In [59]:
stats.levene(강남구,도봉구)

LeveneResult(statistic=np.float64(2178.0436282949113), pvalue=np.float64(0.0))

In [60]:
stats.ttest_ind(도봉구,강남구,equal_var=False)

TtestResult(statistic=np.float64(-54.85483451011352), pvalue=np.float64(0.0), df=np.float64(255877.60839043205))